# Understanding LEGO Sets Popularity: SQL Analysis in Python

## 📖 Background

This project demonstrates SQL querying capabilities within a Python/Jupyter environment using the Rebrickable LEGO database. The analysis explores LEGO set popularity, part usage patterns, and color trends across decades of LEGO production data.

**Technical Environment:** PostgreSQL queries executed via DataCamp Workspace with Python integration

**Analysis Focus:** Understanding product trends, inventory composition, and thematic patterns to support data-driven business insights for stakeholder presentations.

**Dataset Source:** [Rebrickable.com](https://rebrickable.com/downloads) - Comprehensive LEGO database with 11,673 sets and associated parts, colors, and themes.

## 💾 Database Schema

The database contains 8 interconnected tables tracking LEGO inventory at granular levels:

### Core Tables

#### **sets**
- `set_num` - unique set identifier (PK)
- `name` - set name
- `year` - release year
- `theme_id` - FK to themes table
- `num_parts` - total part count per set

#### **inventories**
- `id` - inventory identifier (PK)
- `version` - version number for set variations
- `set_num` - FK to sets table

### Part Hierarchy

#### **parts**
- `part_num` - unique part identifier (PK)
- `name` - part name/description
- `part_cat_id` - FK to part_categories

#### **part_categories**
- `id` - category identifier (PK)
- `name` - category name (e.g., "Bricks", "Plates")

### Inventory Details

#### **inventory_parts**
- `inventory_id` - FK to inventories (PK component)
- `part_num` - FK to parts (PK component)
- `color_id` - FK to colors
- `quantity` - number of parts included
- `is_spare` - boolean flag for spare parts

#### **inventory_sets**
- `inventory_id` - FK to inventories
- `set_num` - FK to sets
- `quantity` - number of sets in inventory

### Reference Data

#### **colors**
- `id` - color identifier (PK)
- `name` - color name
- `rgb` - hexadecimal color code
- `is_trans` - transparency indicator

#### **themes**
- `id` - theme identifier (PK)
- `name` - theme name
- `parent_id` - FK for hierarchical theme relationships (self-referential)

### Relationships

- Sets contain inventories (1:many)
- Inventories contain multiple parts with color/quantity specifications (many:many via inventory_parts)
- Parts belong to categories (many:1)
- Themes have hierarchical parent-child relationships (self-referential)
- Inventories can include other sets as components (many:many via inventory_sets)

***Acknowledgments**: Rebrickable.com*

## Entity-Relationship Diagram

![LEGO Database ERD](data/lego_erd.png)

## 1. What is the average number of LEGO sets released per year?

**Business Question:** Understanding production volume trends

**SQL Techniques:** Common Table Expression (CTE), COUNT aggregation, AVG function, GROUP BY

In [1]:
WITH set_count_per_year AS (
    SELECT year, COUNT(set_num) AS set_count
    FROM sets
    GROUP BY year
)
SELECT AVG(set_count) AS avg_sets_per_year
FROM set_count_per_year;

/* This query first uses a Common Table Expression (CTE) 
to group the sets by year and count the number of sets per year. 
Then it uses the AVG function to calculate the average number of 
sets across all the years. */

,avg_sets_per_year
0,176.863636


## 2. What is the average number of LEGO parts per year?

**Business Question:** Analyzing set complexity trends over time

**SQL Techniques:** AVG aggregation, GROUP BY, ORDER BY with multiple columns (DESC, ASC)

In [1]:
SELECT 
    year, 
    AVG(num_parts) as avg_num_parts 
FROM sets 
GROUP BY year
ORDER BY AVG(num_parts) DESC, year;

/* The query will return the average number of Lego parts 
for each year and it will be ordered by the average number of parts 
in descending order and then by year. */

## 3. Which colors are most popular in LEGO sets?

**Business Question:** Understanding color preferences for inventory planning

**SQL Techniques:** Multi-table JOIN (3 tables), SUM aggregation, GROUP BY, ORDER BY, LIMIT

In [ ]:
SELECT 
    c.name AS color_name, 
    SUM(ip.quantity) AS total_quantity
FROM inventory_parts ip
JOIN inventories i ON ip.inventory_id = i.id
JOIN colors c ON ip.color_id = c.id
GROUP BY c.name
ORDER BY total_quantity DESC
LIMIT 10;

/* This query joins the inventory_parts, inventories, and colors tables 
to calculate the total quantity of each color used across all LEGO sets.
Results are limited to the top 10 most popular colors. */

## 4. What percentage of LEGO parts are transparent?

**Business Question:** Analyzing material composition for manufacturing insights

**SQL Techniques:** Subquery, conditional aggregation with CASE WHEN, percentage calculation with CAST, ROUND function

In [ ]:
SELECT 
    ROUND(
        (SELECT SUM(ip.quantity) 
         FROM inventory_parts ip 
         JOIN colors c ON ip.color_id = c.id 
         WHERE c.is_trans = 't') * 100.0 / 
        (SELECT SUM(ip.quantity) 
         FROM inventory_parts ip),
        2
    ) AS transparent_percentage;

/* This query calculates the percentage of transparent parts by dividing 
the sum of transparent part quantities by the total quantity of all parts,
multiplying by 100, and rounding to 2 decimal places. */

## 5. What are the rarest LEGO parts?

**Business Question:** Identifying limited-edition or discontinued components

**SQL Techniques:** Multi-table JOIN, SUM aggregation, GROUP BY, HAVING clause, ORDER BY, LIMIT

In [ ]:
SELECT 
    p.name, 
    SUM(ip.quantity) AS total_quantity
FROM inventory_parts ip
JOIN parts p ON ip.part_num = p.part_num
GROUP BY p.name
HAVING SUM(ip.quantity) = 1
ORDER BY total_quantity ASC
LIMIT 10;

/* This query identifies the rarest LEGO parts by finding parts with 
a total quantity of exactly 1 across all sets. The HAVING clause filters 
for these unique parts after aggregation. */

## 6. What are the most common LEGO parts across all sets?

**Business Question:** Understanding core inventory components for supply chain optimization

**SQL Techniques:** Multi-table JOIN, SUM aggregation, GROUP BY, ORDER BY DESC, TOP-N query with LIMIT

In [ ]:
SELECT 
    p.name, 
    SUM(ip.quantity) AS total_quantity
FROM inventory_parts ip
JOIN parts p ON ip.part_num = p.part_num
GROUP BY p.name
ORDER BY total_quantity DESC
LIMIT 10;

/* This query identifies the most frequently used LEGO parts by summing 
quantities across all inventory instances and ordering by total quantity 
in descending order. Limited to top 10 results. */

## 7. Which LEGO themes are most popular based on number of sets?

**Business Question:** Identifying successful product lines for strategic planning

**SQL Techniques:** JOIN operation, COUNT aggregation, GROUP BY, ORDER BY DESC, TOP-N analysis

In [ ]:
SELECT 
    t.name AS theme_name, 
    COUNT(s.set_num) AS num_sets
FROM sets s
JOIN themes t ON s.theme_id = t.id
GROUP BY t.name
ORDER BY num_sets DESC
LIMIT 10;

/* This query analyzes theme popularity by counting the number of sets 
in each theme category. Results are ordered by set count to identify 
the most prolific themes. */

## Analysis Summary & Business Insights

### Key Findings

#### 1. Production Volume Trends
- **Average annual releases:** 177 sets per year across the dataset
- **Set complexity evolution:** Peak complexity occurred in 2017 with an average of 261 parts per set
- **Historical baseline:** 1990 showed the lowest complexity at 202 parts per set
- **Business implication:** Increasing set complexity suggests a trend toward more sophisticated, collector-oriented products

#### 2. Color Usage Patterns
- **Dominant colors:** Black leads with 115,085 parts, followed by White, Light Bluish Gray, Red, and Dark Bluish Gray
- **Neutral palette dominance:** Top 5 colors include 3 neutral tones (black, white, grays), suggesting versatile building applications
- **Transparency prevalence:** 6.3% of all parts are transparent, indicating specialized use cases (windows, lights, water elements)
- **Supply chain insight:** High concentration in neutral colors allows for efficient inventory management and production economies of scale

#### 3. Part Rarity Analysis
**Rarest components identified:**
- Slope Curved 2 x 2 x 2/3 with Wolf Legend Beast Eyes
- B-wing Cockpit Print variations
- Jake the Dog Face Print specialized pieces

**Rarity characteristics:**
- All identified rare parts have quantities of 1 across the entire dataset
- Limited production runs or discontinued status
- Theme-specific decorative elements (licensed properties, special editions)
- **Collector value:** These unique parts drive secondary market interest and set collectibility

#### 4. Core Building Components
**Most common parts:**
1. Brick 1 x 2 (69,250 units)
2. Plate 1 x 2 (58,777 units)
3. Round Plate 1 x 1 with Solid Stud
4. Brick 2 x 2
5. Brick 1 x 1

**Strategic insights:**
- Basic rectangular bricks and plates form the foundation of LEGO inventory
- Small connectors (1x1, 1x2) are essential across all set types
- High-volume parts enable modular design and cross-set compatibility
- **Production efficiency:** Concentrating on high-volume basics reduces manufacturing complexity

#### 5. Theme Popularity Analysis
**Top 5 themes by set count:**
1. **Supplemental** - 496 sets (expansion packs, additional parts)
2. **Technic** - 435 sets (mechanical/technical builds)
3. **City** - 287 sets (urban environments)
4. **Friends** - 269 sets (lifestyle/social themes)
5. **Basic Set** - 257 sets (fundamental building blocks)

**Market segmentation insights:**
- **Supplemental theme leadership:** Indicates strong aftermarket demand for expansion options
- **Technic prominence:** Appeals to advanced builders and adult enthusiasts
- **Lifestyle themes (City/Friends):** Capture narrative play and diverse demographic segments
- **Basic sets remain relevant:** Foundation for entry-level builders and creative play

### SQL Technical Competencies Demonstrated

This analysis showcases proficiency in executing SQL within a Python environment (Jupyter/DataCamp Workspace), demonstrating skills directly applicable to customer experience analytics:

**Advanced SQL Techniques:**
- **Common Table Expressions (CTEs):** Used for multi-step calculations and improved query readability
- **Multi-table JOINs:** Successfully joined up to 3 tables to access distributed data
- **Aggregate functions:** AVG, SUM, COUNT for quantitative analysis
- **Subqueries:** Nested queries for percentage calculations and filtered aggregations
- **Conditional logic:** CASE WHEN statements for categorization
- **HAVING clause:** Post-aggregation filtering for rarity analysis
- **TOP-N queries:** LIMIT clause for focused result sets
- **Multi-column sorting:** ORDER BY with DESC/ASC for prioritized results

**Python-SQL Integration:**
- Executing PostgreSQL queries through Python environment
- Managing database connections in notebook workflows
- DataFrame output handling for subsequent analysis
- Professional documentation with markdown annotations

**Business Analysis Skills:**
- Translating business questions into SQL logic
- Identifying relevant metrics for stakeholder presentations
- Data-driven insight generation from relational databases
- Strategic recommendations based on quantitative findings

---

**Version:** 1.0  
**Author:** Rodion  
**Date:** 2023  
**Platform:** DataCamp Workspace  
**Database:** PostgreSQL (Rebrickable LEGO Database)


![Lego Analysis](Lego%20(I).jpg)